In [1]:
import pathlib
import os
if not "changed_working_directory" in locals():
    changed_working_directory = True
    os.chdir(pathlib.Path().resolve().parent)
    
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

In [2]:
from tabGAN import TabGAN, TabGAN2
from src import constants as const
import utils
from utils.tictoc import *
from utils.timer import Timer

imports_path = "src//imports.ipynb"
%run "$imports_path"

if const.dir.project() != os.getcwd():
    raise ValueError("Project directory not the same as in consts")

In [3]:
dataset_train_path = os.path.join(const.dir.data(), "df_adult_edited_train.csv")
dataset_test_path = os.path.join(const.dir.data(), "df_adult_edited_test.csv")

data_train = pd.read_csv(dataset_train_path)
data_test = pd.read_csv(dataset_test_path)
discrete_columns = data_train.columns[data_train.dtypes == "object"]

In [4]:
n_epochs = 100
n_critic = 10
opt_lr = 0.002
adam_beta1 = 0.5
noise_discrete_unif_max = 0

batch_size = 500
ckpt_every = 1
loss_plot_update_every = 10

timer_path = os.path.join(const.dir.timers(), "timer_tabGAN_speed.pkl")
retrain = True

if not os.path.exists(timer_path):
    timer = utils.timer.Timer()
    timer.save(timer_path)
    del timer

In [5]:
@tf.function(experimental_compile=True)
def add(a, b):
    return a+b

add(5,2)

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [ ]:
tf_data_use=False
tf_data_prefetch=False
tf_data_cache=False
tf_make_graph=False
jit_compile=False

tg_qtr = TabGAN(data_train, n_critic = n_critic, opt_lr = opt_lr, adam_beta1 = adam_beta1,
                quantile_transformation_int = True, quantile_rand_transformation = True,
                noise_discrete_unif_max = noise_discrete_unif_max,
                tf_data_use=tf_data_use, tf_data_prefetch=tf_data_prefetch, tf_data_cache=tf_data_cache,
                tf_make_graph=tf_make_graph, jit_compile=jit_compile)

timer_key = "".join((f"tf_data_use={tf_data_use}, tf_data_prefetch={tf_data_prefetch}, tf_data_cache={tf_data_cache}, ",
                    f"tf_make_graph={tf_make_graph}, jit_compile={jit_compile}"
                    ))
if retrain:
    timer_tabGAN = utils.timer.load(timer_path)
    timer_tabGAN.start(timer_key)
    tg_qtr.train(n_epochs, batch_size = batch_size, restart_training = True, progress_bar=True)
    timer_tabGAN.stop(timer_key)
    timer_tabGAN.save(timer_path)

In [14]:
tf_data_use=False
tf_data_prefetch=False
tf_data_cache=False
tf_make_graph=True
jit_compile=False

tg_qtr = TabGAN(data_train, n_critic = n_critic, opt_lr = opt_lr, adam_beta1 = adam_beta1,
                quantile_transformation_int = True, quantile_rand_transformation = True,
                noise_discrete_unif_max = noise_discrete_unif_max,
                tf_data_use=tf_data_use, tf_data_prefetch=tf_data_prefetch, tf_data_cache=tf_data_cache,
                tf_make_graph=tf_make_graph, jit_compile=jit_compile)

timer_key = "".join((f"tf_data_use={tf_data_use}, tf_data_prefetch={tf_data_prefetch}, tf_data_cache={tf_data_cache}, ",
                    f"tf_make_graph={tf_make_graph}, jit_compile={jit_compile}"
                    ))
if retrain:
    timer_tabGAN = utils.timer.load(timer_path)
    timer_tabGAN.start(timer_key)
    tg_qtr.train(n_epochs, batch_size = batch_size, restart_training = True, progress_bar=True)
    timer_tabGAN.stop(timer_key)
    timer_tabGAN.save(timer_path)

  0%|          | 0/100 [00:00<?, ?it/s]

Elapsed time for tf_data_use=False, tf_data_prefetch=False, tf_data_cache=False, tf_make_graph=True, jit_compile=False: 569.120 seconds


In [20]:
tg_qtr = TabGAN(data_train, n_critic = n_critic, opt_lr = opt_lr, adam_beta1 = adam_beta1,
                quantile_transformation_int = True, quantile_rand_transformation = True,
                noise_discrete_unif_max = noise_discrete_unif_max, jit_compile_train_step=True)

if retrain:
    timer_tabGAN = utils.timer.load(timer_path)
    timer_tabGAN.start("jit_compile=True")
    tg_qtr.train(n_epochs, batch_size = batch_size, restart_training = True,
             progress_bar=True)
    timer_tabGAN.stop("jit_compile=True")
    timer_tabGAN.save(timer_path)

  0%|          | 0/100 [00:00<?, ?it/s]

Elapsed time for jit_compile=True: 436.841 seconds


In [18]:
timer_tabGAN = utils.timer.load(timer_path)
timer_tabGAN.elapsed_time["jit_compile=True"] / timer_tabGAN.elapsed_time["jit_compile=False"]

0.7518136797897509

In [21]:
tg_qtr2 = TabGAN2(data_train, n_critic = n_critic, opt_lr = opt_lr, adam_beta1 = adam_beta1,
                quantile_transformation_int = True, quantile_rand_transformation = True,
                noise_discrete_unif_max = noise_discrete_unif_max, jit_compile_train_step=False,
                 tf_make_train_step_graph=False)

if retrain:
    timer_tabGAN = utils.timer.load(timer_path)
    timer_tabGAN.start("tabGAN2, jit_compile=False")
    %lprun -f tg_qtr2.train_step_func tg_qtr2.train(n_epochs, batch_size = batch_size, restart_training = True, progress_bar=True)
    timer_tabGAN.stop("tabGAN2, jit_compile=False")
    timer_tabGAN.save(timer_path)

  0%|          | 0/100 [00:00<?, ?it/s]

Elapsed time for tabGAN2, jit_compile=False: 334.257 seconds


In [35]:
class ab():
    def __init__(self, hei):
        print(hei)
        self.hei = True
    @tf.function(experimental_compile=self.hei)
    def p():
        print("Jaaaaaa")

NameError: name 'self' is not defined

In [27]:
tg_qtr2 = TabGAN2(data_train, n_critic = n_critic, opt_lr = opt_lr, adam_beta1 = adam_beta1,
                quantile_transformation_int = True, quantile_rand_transformation = True,
                noise_discrete_unif_max = noise_discrete_unif_max, jit_compile_train_step=False,
                 tf_make_train_step_graph=False, use_tf_data=False)

if retrain:
    timer_tabGAN = utils.timer.load(timer_path)
    timer_tabGAN.start("tabGAN2, jit_compile=False")
    %lprun -f tg_qtr2.train_step_func tg_qtr2.train(n_epochs, batch_size = batch_size, restart_training = True, progress_bar=True)
    timer_tabGAN.stop("tabGAN2, jit_compile=False")
    timer_tabGAN.save(timer_path)

  0%|          | 0/100 [00:00<?, ?it/s]

Elapsed time for tabGAN2, jit_compile=False: 303.208 seconds


In [28]:
tg_qtr2 = TabGAN2(data_train, n_critic = n_critic, opt_lr = opt_lr, adam_beta1 = adam_beta1,
                quantile_transformation_int = True, quantile_rand_transformation = True,
                noise_discrete_unif_max = noise_discrete_unif_max, jit_compile_train_step=False)

tic()
tg_qtr2.train(n_epochs, batch_size = batch_size, restart_training = True, progress_bar=True)
toc()

  0%|          | 0/100 [00:00<?, ?it/s]

Elapsed time: 327.493996 seconds.



In [15]:
tg_qtr2 = TabGAN2(data_train, n_critic = n_critic, opt_lr = opt_lr, adam_beta1 = adam_beta1,
                quantile_transformation_int = True, quantile_rand_transformation = True,
                noise_discrete_unif_max = noise_discrete_unif_max, jit_compile_train_step=False)

tic()
tg_qtr2.train(n_epochs, batch_size = batch_size, restart_training = True, progress_bar=True)
toc()

  0%|          | 0/100 [00:00<?, ?it/s]

Elapsed time: 574.917239 seconds.



In [16]:
tg_qtr2 = TabGAN2(data_train, n_critic = n_critic, opt_lr = opt_lr, adam_beta1 = adam_beta1,
                quantile_transformation_int = True, quantile_rand_transformation = True,
                noise_discrete_unif_max = noise_discrete_unif_max, jit_compile_train_step=False)

tic()
tg_qtr2.train(n_epochs, batch_size = batch_size, restart_training = True, progress_bar=True)
toc()

  0%|          | 0/100 [00:00<?, ?it/s]

Elapsed time: 572.547649 seconds.



In [18]:
tg_qtr2 = TabGAN2(data_train, n_critic = n_critic, opt_lr = opt_lr, adam_beta1 = adam_beta1,
                quantile_transformation_int = True, quantile_rand_transformation = True,
                noise_discrete_unif_max = noise_discrete_unif_max, jit_compile_train_step=False)

tic()
tg_qtr2.train(n_epochs, batch_size = batch_size, restart_training = True, progress_bar=True)
toc()

  0%|          | 0/100 [00:00<?, ?it/s]

Elapsed time: 355.904017 seconds.



In [19]:
356 / 572

0.6223776223776224